# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 86 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.13132


extracting tarball to tmp_2404.13132...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2404.13132/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Retrieving document from  https://arxiv.org/e-print/2404.14127


extracting tarball to tmp_2404.14127...

 done.


T. Cantat-Gaudin  ->  T. Cantat-Gaudin  |  ['T. Cantat-Gaudin']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 36 bibliographic references in tmp_2404.14127/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.14127-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.14127) | **Gaia DR3 detectability of unresolved binary systems**  |
|| A. Castro-Ginard, et al. -- incl., <mark>T. Cantat-Gaudin</mark>, <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-04-23*|
|*Comments*| *11 pages, 10 figures. Accepted for publication in A&A*|
|**Abstract**| Gaia can not individually resolve very close binary systems, however, the collected data can still be used to identify them. A powerful indicator of stellar multiplicity is the sources reported Renormalized Unit Weight Error (ruwe), which effectively captures the astrometric deviations from single-source solutions. We aim to characterise the imprints left on ruwe caused by binarity. By flagging potential binary systems based on ruwe, we aim to characterise which of their properties will contribute the most to their detectability. We develop a model to estimate ruwe values for observations of Gaia sources, based on the biases to the single-source astrometric track arising from the presence of an unseen companion. Then, using the recipes from previous GaiaUnlimited selection functions, we estimate the selection probability of sources with high ruwe, and discuss what binary properties contribute to increasing the sources ruwe. We compute the maximum ruwe value which is compatible with single-source solutions as a function of their location on-sky. We see that binary systems selected as sources with a ruwe higher than this sky-varying threshold have a strong detectability window in their orbital period distribution, which peaks at periods equal to the Gaia observation time baseline. We demonstrate how our sky-varying ruwe threshold provides a more complete sample of binary systems when compared to single sky-averaged values by studying the unresolved binary population in the Gaia Catalogue of Nearby Stars. We provide the code and tools used in this study, as well as the sky-varying ruwe threshold through the GaiaUnlimited Python package |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.13132-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.13132) | **Medium Bands, Mega Science: a JWST/NIRCam Medium-Band Imaging Survey of  Abell 2744**  |
|| K. A. Suess, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-04-23*|
|*Comments*| *21 pages, 10 figures. Fully reduced imaging, photometric catalogs, and photometric redshift fits publicly available at this https URL*|
|**Abstract**| In this paper, we describe the "Medium Bands, Mega Science" JWST Cycle 2 survey (JWST-GO-4111) and demonstrate the power of these data to reveal both the spatially-integrated and spatially-resolved properties of galaxies from the local universe to the era of cosmic dawn. Executed in November 2023, MegaScience obtained ~30 arcmin^2 of deep multiband NIRCam imaging centered on the z~0.3 Abell 2744 cluster, including eleven medium-band filters and the two shortest-wavelength broad-band filters, F070W and F090W. Together, MegaScience and the UNCOVER Cycle 1 treasury program provide a complete set of deep (~28-30 mag) images in all NIRCam medium- and broad-band filters. This unique dataset allows us to precisely constrain photometric redshifts, map stellar populations and dust attenuation for large samples of distant galaxies, and examine the connection between galaxy structures and formation histories. MegaScience also includes ~17 arcmin^2 of NIRISS parallel imaging in two broad-band and four medium-band filters from 0.9-4.8um, expanding the footprint where robust spectral energy distribution (SED) fitting is possible. We provide example SEDs and multi-band cutouts at a variety of redshifts, and use a catalog of JWST spectroscopic redshifts to show that MegaScience improves both the scatter and catastrophic outlier rate of photometric redshifts by factors of 2-3. Additionally, we demonstrate the spatially-resolved science enabled by MegaScience by presenting maps of the [OIII] line emission and continuum emission in three spectroscopically-confirmed z>6 galaxies. We show that line emission in reionization-era galaxies can be clumpy, extended, and spatially offset from continuum emission, implying that galaxy assembly histories are complex even at these early epochs. We publicly release fully reduced mosaics and photometric catalogs for both the NIRCam primary and NIRISS parallel fields. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 36</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.14127.md
    + _build/html/tmp_2404.14127/./figures/RUWE_threshold_singleSource.png
    + _build/html/tmp_2404.14127/./figures/RUWE_SF.png
    + _build/html/tmp_2404.14127/./figures/detectability_cornerPlot_reduced.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\Gaia}{\textit{Gaia}\xspace}$
$\newcommand{\ruwe}{\texttt{ruwe}\xspace}$
$\newcommand{\cat}{\ensuremath{\mathcal{C}}}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# $\Gaia$ DR3 detectability of unresolved binary systems

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.14127-b31b1b.svg)](https://arxiv.org/abs/2404.14127)<mark>Appeared on: 2024-04-23</mark> -  _11 pages, 10 figures. Accepted for publication in A&A_

</div>
<div id="authors">

A. Castro-Ginard, et al. -- incl., <mark>T. Cantat-Gaudin</mark>, <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** $\Gaia$ can not individually resolve very close binary systems, however, the collected data can still be used to identify them. A powerful indicator of stellar multiplicity is the sources reported Renormalized Unit Weight Error ( $\ruwe$ ), which effectively captures the astrometric deviations from single-source solutions. We aim to characterise the imprints left on $\ruwe$ caused by binarity. By flagging potential binary systems based on $\ruwe$ , we aim to characterise which of their properties will contribute the most to their detectability. We develop a model to estimate $\ruwe$ values for observations of $\Gaia$ sources, based on the biases to the single-source astrometric track arising from the presence of an unseen companion. Then, using the recipes from previous GaiaUnlimited selection functions, we estimate the selection probability of sources with high $\ruwe$ , and discuss what binary properties contribute to increasing the sources' $\ruwe$ . We compute the maximum $\ruwe$ value which is compatible with single-source solutions as a function of their location on-sky. We see that binary systems selected as sources with a $\ruwe$ higher than this sky-varying threshold have a strong detectability window in their orbital period distribution, which peaks at periods equal to the $\Gaia$ observation time baseline. We demonstrate how our sky-varying $\ruwe$ threshold provides a more complete sample of binary systems when compared to single sky-averaged values by studying the unresolved binary population in the $\Gaia$ Catalogue of Nearby Stars. We provide the code and tools used in this study, as well as the sky-varying $\ruwe$ threshold through the GaiaUnlimited Python package.

</div>

<div id="div_fig1">

<img src="tmp_2404.14127/./figures/RUWE_threshold_singleSource.png" alt="Fig9" width="100%"/>

**Figure 9. -** Sky map of the sky-varying $\ruwe$ threshold at the resolution of HEALPix level 5. Sources above these $\ruwe$ threshold indicate potential binary systems. These $\ruwe$ values show a strong influence of the $\Gaia$ scanning law, while also highlighting regions with high source density. (*fig:ruwe_singleSource*)

</div>
<div id="div_fig2">

<img src="tmp_2404.14127/./figures/RUWE_SF.png" alt="Fig3" width="100%"/>

**Figure 3. -** Selection probability for sources with a $\ruwe$ higher than the threshold represented in Fig. \ref{fig:ruwe_singleSource}. The regions with higher selection probability, e.g., cores of globular clusters, the Galactic plane and the Magellanic Clouds, contain sources with a $\ruwe$ increased due to a combination of binarity, source crowding and other effects. These regions may indicate a higher contamination rate when selecting binaries solely based on $\ruwe$. The dark halos around the Magellanic Clouds are due to an excess of faint sources with low $\ruwe$ in that region. (*fig:binaries_SF*)

</div>
<div id="div_fig3">

<img src="tmp_2404.14127/./figures/detectability_cornerPlot_reduced.png" alt="Fig4" width="100%"/>

**Figure 4. -** Heatmap of binary systems' properties, i.e., distance, period and mass rations, that will enhance their detection. The detectability of these systems monotonically decreases with distance, reaching a detection fraction of $50\%$ at around $100$ pc. There is a strong detectability window around periods of three years, corresponding to the $\Gaia$ observation time baseline. Systems with mass ratios of zero, i.e., single sources, or one show no sign of binarity on $\ruwe$ since there is no wobbling of their CoM around the photocentre. (*fig:detectability_cornerPlot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.14127"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

410  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
